In [1]:
import pandas as pd
import seaborn as sns
import plotly
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from collections import defaultdict


In [2]:
def contains_shir(row):
    # exception
    if row[email] == 'kshirjarohannaik@gmail.com':
        return False
    elif 'shir' in row[email]:
        return True
    elif 'shir' in row[name]:
        return True
    else:
        return False

In [3]:
df = pd.read_csv("../data/subscriptions.csv")
email = 'Customer Email'
name = 'Customer Name'
c_id = 'Customer ID'

df[email] = df[email].str.lower()
df[name] = df[name].str.lower()


# no empty values
print(f"{df[email].isna().sum() = }")
print(f"{df[name].isna().sum() = }")

df[email].isna().sum() = 0
df[name].isna().sum() = 0


In [4]:
shir_mask = df.apply(contains_shir, axis=1)
df = df[~shir_mask]

In [9]:
def assign_duplicate_group_ids(df, email_col, name_col):
    df = df.copy()  # do not mutate caller
    # Ensure positional alignment 0..n-1
    df = df.reset_index(drop=True)
    n = len(df)

    # Group labels as arrays (positional)
    e_group = df.groupby(email_col).ngroup().to_numpy()
    n_group = df.groupby(name_col).ngroup().to_numpy()

    # Build adjacency by shared email OR shared name
    email_to_rows = defaultdict(list)
    name_to_rows  = defaultdict(list)
    for i in range(n):
        email_to_rows[e_group[i]].append(i)
        name_to_rows[n_group[i]].append(i)

    # DFS over rows (0..n-1), using only positional indices
    visited = set()
    group_id = [-1] * n
    current_gid = 0

    for start in range(n):
        if start in visited:
            continue
        stack = [start]
        while stack:
            cur = stack.pop()
            if cur in visited:
                continue
            visited.add(cur)
            group_id[cur] = current_gid

            # push all rows sharing this row's email
            for nbr in email_to_rows[e_group[cur]]:
                if nbr not in visited:
                    stack.append(nbr)
            # push all rows sharing this row's name
            for nbr in name_to_rows[n_group[cur]]:
                if nbr not in visited:
                    stack.append(nbr)

        current_gid += 1

    df["group_id"] = group_id
    return df


In [12]:
dups = assign_duplicate_group_ids(df, email_col=email, name_col=name)

In [34]:
 collapse_duplicate_groups(
    df: pd.DataFrame,
    group_col: str = "group_id",
    min_col: str = "column 1",
    max_col: str = "column 2",
) 

,id,Customer ID,Customer Email,Quantity,Amount,Status,Created (UTC),Start (UTC),Start Date (UTC),Current Period Start (UTC),Current Period End (UTC),Trial Start (UTC),Trial End (UTC),Canceled At (UTC),Cancel At Period End,Ended At (UTC),Customer Name
241,sub_1Os5xVLLDYopDCcGmS9xAiVJ,cus_PhUxQjYi7GgETP,mertiti@gmail.com,1,129.0,canceled,2024-03-08 16:07,2024-03-08 16:07,2024-03-08 16:07,2024-06-08 16:07,2024-07-08 16:07,NaN,NaN,2024-06-21 21:52,False,2024-06-21 21:52,avinoam gal


In [8]:
dups